# Classifying Claims - Getting the Data

Here are a short series of notebooks that investigate classifying patent claims using deep learning techniques.

In this post we will generate a data source to use for classifying patent claims.

We will be using USPTO data, where the claims are classified according to the International Patent Classification (IPC). To keep things simple we will use the first letter of the IPC (top level category). This is the same as the top level of the Cooperative Patent Classification (CPC).  

The list of top level categories can be found here: https://rs.espacenet.com/help?locale=en_EP&method=handleHelpTopic&topic=ipc:
* A Human Necessities
* B Performing Operations; Transporting
* C Chemistry; Metallurgy
* D Textiles; Paper
* E Fixed Constructions
* F Mechanical Engineering; Lighting; Heating; Weapons; Blasting Engines or Pumps
* G Physics
* H Electricity

---
## Getting Our Data

Most of the task revolves around getting our data and placing it in a form where we can apply common machine learning libraries.  

In [1]:
# Start with getting 12,000 patent publications at random
from patentdata.corpus import USPublications
from patentdata.models.patentcorpus import LazyPatentCorpus
import os, pickle

from collections import Counter

We have some function that provide a wrapper around USPTO patent publication data as obtained from the bulk data download page (https://www.uspto.gov/learning-and-resources/bulk-data-products). This code can be found on GitHub - https://github.com/benhoyle/patentdata.

The code below gets 12,000 random patent publications from data for years 2001 to 2017 across all classifications. From each document the text for claim 1 and the classifications are extracted.

In [2]:
# Get the claim 1 and classificationt text

PIK = "claim_and_class.data"

if os.path.isfile(PIK):
    with open(PIK, "rb") as f:
        print("Loading data")
        data = pickle.load(f)
        print("{0} claims and classifications loaded".format(len(data)))
else:
    # Load our list of records
    PIK = "12000records.data"

    if os.path.isfile(PIK):
        with open(PIK, "rb") as f:
            print("Loading data")
            records = pickle.load(f)
            print("{0} records loaded".format(len(records)))
    else:
        path = '/media/SAMSUNG1/Patent_Downloads'
        ds = USPublications(path)
        records = ds.get_records([], "name", sample_size=12000)
        with open(PIK, "wb") as f:
            pickle.dump(records, f)
            print("{0} records saved".format(len(records)))
    
    lzy = LazyPatentCorpus()
    lzy.init_by_filenames(ds, records)
    
    data = list()
    for i, pd in enumerate(lzy.documents):
        try:
            classifications = [c.as_string() for c in pd.classifications]
        except:
            classifications = ""
        try:
            claim1_text = pd.claimset.get_claim(1).text
        except:
            claim1_text = ""
        current_data = (claim1_text, classifications)
        data.append(current_data)
        if (i % 500) == 0:
            print("Saving a checkpoint at {0} files".format(i))
            print("Current data = ", current_data)
            with open(PIK, "wb") as f:
                pickle.dump(data, f)
            
    with open(PIK, "wb") as f:
        pickle.dump(data, f)
        
    print("{0} claims saved".format(len(data)))

Loading data
12000 claims and classifications loaded


We then get rid of claims which are cancelled (these just have "(canceled)" as text).

In [3]:
# Check for and remove 'cancelled' claims
no_cancelled = [d for d in data if '(canceled)' not in d[0]]

print("There are now {0} claims after filtering out cancelled claims".format(len(no_cancelled)))

There are now 11239 claims after filtering out cancelled claims


Then we filter the data to extract the first level classification. For simplicity we take the first classification (if there are several classifications) where the data exists. An example data entry is set out below.

In [4]:
# Get classification in the form of A-H
cleaner_data = list()
for d in no_cancelled:
    if len(d[1]) >= 1:
        if len(d[1][0]) > 3:
            classification = d[1][0][2]
            cleaner_data.append(
                (d[0], classification)
            )

print("There are now {0} claims after extracting classifications".format(len(cleaner_data)))
cleaner_data[55]

There are now 11238 claims after extracting classifications


('\n1. A sensing assembly for sensing a level of liquid in a reservoir, said sensing assembly comprising: \na first input port for receiving a first input voltage signal; \na second input port for receiving a second input voltage signal; \nan excitation circuit electrically connected to said first and second input ports for receiving the first and second input voltage signals and for generating a first excitation signal and a second excitation signal, said excitation circuit includes first and second excitation electrodes extending along a portion of the reservoir, said first and second excitation electrodes disposed adjacent to and separated by said first receiving electrode; and \na receiving circuit disposed adjacent said excitation circuit defining a variable capacitance with said excitation circuit, wherein said receiving circuit includes first and second receiving electrodes extending along a portion of the reservoir and a first trace connected to ground and extending between sai

It is good to start by getting a feel for how the classifications are distributed across the dataset.

In [5]:
# Let's check how our data is distributed across the classes
class_count = Counter([d[1] for d in cleaner_data])
class_count

Counter({'A': 1777,
         'B': 1449,
         'C': 865,
         'D': 54,
         'E': 269,
         'F': 735,
         'G': 3335,
         'H': 2754})

What is quite interesting is we see that the data is mainly clusted around class A, B, G and H. Classes C, D, E and F have a limited number of associated claims.

To help further processing we will clean the characters of the text. The patentdata library has a function that replaces certain characters (e.g. different versions of '"' or '-') with a reduced set of common printable characters. 

In [6]:
# Clean the characters in the data to use a reduced set of printable characters
# There is a function in patentdata to do this
from patentdata.models.lib.utils import clean_characters

cleaner_data = [(clean_characters(d[0]), d[1]) for d in cleaner_data]

Finally we will save our claims and classifications as a Pickle file called "raw_data". We can load this quickly in subsequent notebooks to apply classification.

In [7]:
raw_data = cleaner_data
with open("raw_data.pkl", "wb") as f:
    pickle.dump(raw_data, f)